# Se hace un merge entre Premiere League Matches y FIFA Player Stats

# Databases


#### Hay dos bases de datos:

* **FPS:** Fifa players stats 15-20: https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset
* **PML:** Premiere league matches 14/15-19/20: Scraped from https://www.flashscore.com/football/england/premier-league/archive/

## Se importan las librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
from sklearn.metrics import mean_squared_error
import math
import glob

sns.set(style="darkgrid")

## Se importan los archivos CSVs

#### Se importa la temporada Premiere League 2014 - 2015


In [ ]:
li = []
for i in range(15, 14, -1): 
    path = f'C:/Users/ctn24/PycharmProjects/ScrapeFlashScore/Scripts/Premier_League20{i-1}20{i}/' # use your path
    all_files = glob.glob(path + "/*.csv")
    print(f'Premier_League20{i-1}20{i}')
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

PLM_1415 = pd.concat(li, axis=0, ignore_index=True)

#### Se importan los datos de FIFA 2015

In [ ]:
FPS_15 = pd.read_csv('C:/Users/ctn24/PycharmProjects/ScrapeFlashScore/Scripts/FIFA 15-20/players_15.csv')
FPS = FPS_15.copy()
#FPS = FPS[[ 'nationality', 'club','team_jersey_number', 'short_name', 'dob']]
FPS

In [ ]:
#FPS = FPS.dropna()
#FPS

In [ ]:
PLM_1415.info()

In [ ]:
clubs_PLM = PLM_1415[['Club']]
clubs_PLM.info()

In [ ]:
clubs_PLM['Club'].value_counts().head(3)

In [ ]:
unique_clubs_PLM = pd.DataFrame()
unique_clubs_PLM['Club'] = clubs_PLM.Club.drop_duplicates()
unique_clubs_PLM.info()

#### Checamos que los nombres de los equipos (Club) en PLM esten dentro de FPS

In [ ]:
pd.Series(unique_clubs_PLM.Club.isin(FPS.club).values.astype(int), unique_clubs_PLM.Club.values).head()

#### Cambiamos los nombres de los equipos (Club) en la serie "PLM" para que concuerden con FPS usando un diccionario

In [ ]:
di = {'Brighton':'Brighton & Hove Albion','Tottenham':'Tottenham Hotspur',
'Newcastle':'Newcastle United','Wolves':'Wolverhampton Wanderers',
'Norwich':'Norwich City','Manchester Utd':'Manchester United',
'Leicester':'Leicester City','West Ham':'West Ham United',
'Sheffield Utd':'Sheffield United','Cardiff':'Cardiff City',
'Huddersfield':'Huddersfield Town','Swansea':'Swansea City',
'Stoke':'Stoke City','West Brom':'West Bromwich Albion',
'Hull':'Hull City','QPR':'Queens Park Rangers'}

unique_clubs_PLM_renamed = pd.DataFrame()
unique_clubs_PLM_renamed['Club'] = unique_clubs_PLM['Club'].map(di).fillna(unique_clubs_PLM['Club']) 
unique_clubs_PLM_renamed.head(3)

#### Checamos que todos los nombres de los clubs de PLM esten dentro de FPS

In [ ]:
pd.Series(unique_clubs_PLM_renamed.Club.isin(FPS.club).values.astype(int), unique_clubs_PLM_renamed.Club.values).head()

In [ ]:
FPS_c = FPS.copy()
FPS_c = FPS_c[FPS_c.club.isin(unique_clubs_PLM_renamed['Club'])]
FPS_c.head(3)

In [ ]:
PLM_1415_nat = pd.DataFrame()
PLM_1415_nat['nationality'] = PLM_1415.Nationality.drop_duplicates()

In [ ]:
nat_compare = pd.Series(PLM_1415_nat.nationality.isin(FPS.nationality)
                                .values.astype(int), PLM_1415_nat.nationality.values)

nat_compare.value_counts()

#### Paises con nombre diferente en las bd

In [ ]:
nat_compare[nat_compare.eq(0)].head()

#### Cambiamos los nombres de los paises (Nationality) en la serie de "PLM" para que concuerden con FPS usando un diccionario

In [ ]:
di = {'Ireland':'Republic of Ireland','USA':'United States','South Korea':'Korea Republic',
      'Bosnia and Herzegovina':'Bosnia Herzegovina','Curaçao':'Curacao'}

PLM_1415_nat_renamed = pd.DataFrame()
PLM_1415_nat_renamed['nationality'] = PLM_1415_nat['nationality'].map(di).fillna(PLM_1415_nat['nationality']) 
PLM_1415_nat_renamed.head(3)

In [ ]:
nationality_compare = pd.Series(PLM_1415_nat_renamed.nationality.isin(FPS.nationality)
                                .values.astype(int), PLM_1415_nat_renamed.nationality.values)
nationality_compare.head(3)

## Fucionamos las bases de datos

In [ ]:
#pd.merge(a, b, on=['A', 'B'])
PML_i = PLM_1415.copy()
PML_i = pd.DataFrame(PML_i.drop(['Unnamed: 0'], axis=1))
PML_i.head(3)

In [ ]:
di = {'Ireland':'Republic of Ireland','USA':'United States','South Korea':'Korea Republic',
      'Bosnia and Herzegovina':'Bosnia Herzegovina','Curaçao':'Curacao'}

PML_ni = PML_i.copy()
PML_ni_serie = PML_ni['Nationality'].map(di).fillna(PML_ni['Nationality']) 
PML_ni.insert(6, 'nationality', PML_ni_serie, True)
PML_ni = PML_ni.drop(['Nationality'], axis = 1)
PML_ni.head(3)

In [ ]:
PML_ni.head(1)

In [ ]:
di = {'Brighton':'Brighton & Hove Albion','Tottenham':'Tottenham Hotspur',
'Newcastle':'Newcastle United','Wolves':'Wolverhampton Wanderers',
'Norwich':'Norwich City','Manchester Utd':'Manchester United',
'Leicester':'Leicester City','West Ham':'West Ham United',
'Sheffield Utd':'Sheffield United','Cardiff':'Cardiff City',
'Huddersfield':'Huddersfield Town','Swansea':'Swansea City',
'Stoke':'Stoke City','West Brom':'West Bromwich Albion',
'Hull':'Hull City','QPR':'Queens Park Rangers'}

PML_cni = PML_ni.copy()
PML_cni_serie = PML_cni['Club'].map(di).fillna(PML_cni['Club']) 
PML_cni.insert(3, 'club', PML_cni_serie, True)
PML_cni = PML_cni.drop(['Club'], axis=1)
PML_cni.head(3)

In [ ]:
FPS.head(1)
FPS.info()

In [ ]:
#FPS = FPS.astype({
#                    'nationality':str,
#                    'club':str,
#                    'team_jersey_number':int,
#                    'short_name':str,
#                    'dob':str
#                })
FPS.head(1)

In [ ]:
FPS = FPS.rename(columns={'team_jersey_number': 'jersey_num'})
PML_cni = PML_cni.rename(columns={'Jersey_Num': 'jersey_num'})

#### Al haber mas registros que antes en PML, nos damos cuenta que hay valores en FPS que se relacionan con mas de un registro en PLM

In [ ]:
#pd.merge(a, b, on=['A', 'B'])
PML_FPS = pd.merge(PML_cni, FPS, on=['jersey_num', 'club'], how='left')
PML_FPS.info()

In [ ]:
FPS

In [ ]:
duplicated = FPS.duplicated(subset=['jersey_num','club'])

FPS[duplicated].sort_values(by=['club'])

#Ya no hay duplicados en la Premiere League

In [ ]:
PML_cni.loc[(PML_cni['jersey_num'] == 13)  &  (PML_cni['club'] == 'Queens Park Rangers')].head(1)

In [ ]:
#from IPython.core.display import HTML
#display(HTML(PML_FPS.to_html()))

#merge_dif_nat = PML_FPS[['short_name','jersey_num', 'club', 'nationality_x','nationality_y']].query(
 #   'nationality_x != nationality_y').drop_duplicates(subset=['short_name']).sort_values(by=['jersey_num'])
#merge_dif_nat.head(1)

In [ ]:
PML_cni.info()

In [ ]:
#PML_cni = PML_cni.astype({
#                        'Date':str,
#                        'League':str,
#                        'club':str,
#                        'Goals':int,
#                        'Home_Away':str,
#                        'Name':str,
#                        'nationality':str,
#                        'player_link':str,
#                        'jersey_num':int,
#                        'pos_left':int,
#                        'pos_top':int
#                        })
#PML_cni.head()

#### ((Esto es innecesario)) Removemos los "(C)" y "(G)", de capitan y goalkeeper, de la serie "Name" de PML

PML_ncni_serie = PML_cni['Name'].fillna('N/A').str.replace("\(C\)", "")
PML_ncni_serie = PML_ncni_serie.fillna('N/A').str.replace("\(G\)", "")

#PML_ncni_serie.value_counts()

PML_ncni = PML_cni.copy()
PML_ncni = PML_cni.drop(['Name'], axis=1)
PML_ncni.insert(5, 'name', PML_ncni_serie, True)
PML_ncni.head(3)

In [ ]:
PML_FPS.head(2)

In [ ]:
#df.loc[lambda df: df['shield'] == 8]
PML_FPS_drop = PML_FPS.loc[(PML_FPS['player_positions'] != 'GK')].copy() #(Ya funciona se corrigio la bd)
                                                                         #DA 7591 en lugar de 7600
#PML_FPS_drop = PML_FPS.loc[(PML_FPS['team_position'].str.strip() != 'GK')].copy() #DA 7732 en lugar de 7600
#PML_FPS_drop = PML_FPS.loc[(PML_FPS['pos_left'] != -28) & (PML_FPS['pos_left'] != 448)].copy() # Este si da 7600

PML_FPS_drop = PML_FPS_drop[PML_FPS.columns[:82]]
PML_FPS_drop = PML_FPS_drop.dropna(axis=1, how='all',thresh=7590)
PML_FPS_drop = PML_FPS_drop.drop(['player_url','short_name','long_name',
                                  'age','dob','value_eur','wage_eur','international_reputation',
                                  'skill_moves','team_position',
                                  'player_link','nationality_x','nationality_y','sofifa_id','real_face'                              
                                 ],axis=1)
PML_FPS_drop.info(1)
PML_FPS_drop.to_csv('PML_FPS_drop.csv', index=False)


In [ ]:
PML_FPS_drop_clean = PML_FPS_drop.copy()
col = PML_FPS_drop_clean.columns
#print(col[24])

for i in range(18,len(col),1):
    try:
        PML_FPS_drop_clean[col[i]] = PML_FPS_drop_clean[col[i]].str.split(pat='+').str[0]
        PML_FPS_drop_clean[col[i]] = PML_FPS_drop_clean[col[i]].str.split(pat='-').str[0]
    except:
        next     
    PML_FPS_drop_clean[col[i]] = PML_FPS_drop_clean[col[i]].astype(int)

    #print(PML_FPS_drop_clean[col[i]])

PML_FPS_drop_clean.info()

#PML_FPS_att = PML_FPS['attacking_crossing'].str.split(pat='+').str[0]
#PML_FPS_att = PML_FPS_att.str.split(pat='-').str[0]
#PML_FPS_att

In [ ]:
PML_FPS_drop_clean.describe()

In [ ]:
PML_FPS_pmap = PML_FPS_drop_clean.copy()
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 11] = 12
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 20] = 21
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 75] = 76
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 81] = 82
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 100] = 101
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 138] = 142
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 183] = 184
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 200] = 201
#Como por error se mezclan las posiciones en los extremos cuando son 4 y 5 entonces las fucionamos como una sola.
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 202] = 203
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 264] = 263
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 12] = 21
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 265] = 263

In [ ]:
PML_FPS_pmap['pos_top'].sort_values().value_counts()

#### Aqui (pos_top) mejor dividi el campo dependiendo de la formacion y junte los que deberian ser iguales, e.g. 1/3 con 2/6

In [ ]:
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 76] = 82
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 203] = 200
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 201] = 200

PML_FPS_pmap['pos_top'].sort_values().value_counts()

In [ ]:
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 21] = 0
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 62] = 1
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 82] = 2
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 101] = 3
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 142] = 4
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 184] = 5
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 200] = 6
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 222] = 7
PML_FPS_pmap['pos_top'].loc[PML_FPS_pmap['pos_top'] == 263] = 8
PML_FPS_pmap['pos_top'].value_counts()

In [ ]:
PML_FPS_drop_clean['pos_left'].sort_values(axis=0).drop_duplicates()

In [ ]:
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 18] = 0
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 34] = 1
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 69] = 2
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 104] = 3
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 120] = 4
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 174] = 5

PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 402] = 11
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 386] = 10
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 351] = 9
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 316] = 8
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 300] = 7
PML_FPS_pmap['pos_left'].loc[PML_FPS_pmap['pos_left'] == 246] = 6
PML_FPS_pmap['pos_left'].value_counts()

In [ ]:
(PML_FPS_pmap.query
 ('League == "Premier League - Round 10" and Date == "01.11.2014 09:00" and (club == "Arsenal" or club == "Burnley")')
 .plot.scatter(x='pos_left',y='pos_top',c='movement_acceleration',colormap='viridis'))



In [ ]:
#PML_FPS.plot.scatter(x='pos_left', y='pos_top', c='DarkBlue')

In [ ]:
(PML_FPS.query('League == "Premier League - Round 10" and (club == "Arsenal" or club == "Burnley")')
 .plot.scatter(x='pos_left',y='pos_top',c='overall',colormap='viridis'))


In [ ]:
# (PML_FPS.query('League == "Premier League - Round 10" and (club == "Manchester City" or club == "Manchester United")')
# .plot.scatter(x='pos_left',y='pos_top',c='overall',colormap='viridis'))

In [ ]:
PML_FPS.query('League == "Premier League - Round 10"').drop_duplicates('club').head(1)

### Convertimos los datos categoricos a numericos

In [ ]:
PML_FPS_pmap['work_rate'].drop_duplicates()

In [ ]:
# new data frame with split value columns 
new = PML_FPS_pmap['work_rate'].str.split("/", n = 1, expand = True) 
  
# making separate first name column from new data frame 
PML_FPS_pmap["work_att"]= new[0] 
  
# making separate last name column from new data frame 
PML_FPS_pmap["work_def"]= new[1] 
  
# Dropping old Name columns 
#PML_FPS_pmap.drop(columns =["work_rate"], inplace = True) 
  
# df display 
PML_FPS_pmap.head(1)

In [ ]:
work_rates = PML_FPS_pmap[['work_att','work_def']].copy()
work_rates.tail(3)

#### No usamos el OrdinalEncoder ya que pone los valores por abecedario y optamos por mapearlo con un diccionario

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

In [ ]:
work_rates_encoded =  ordinal_encoder.fit_transform(work_rates)
work_rates_encoded

In [ ]:
ordinal_encoder.categories_

In [ ]:
mapping = { "Low" : 0,"Medium" : 1, "High" : 2}

work_rates['int_work_att'] = work_rates[['work_att']].applymap(lambda x : mapping[x])
work_rates['int_work_def'] = work_rates[['work_def']].applymap(lambda x : mapping[x])

work_rates = work_rates.drop(columns =["work_att",'work_def'])
work_rates.info()

In [ ]:
PML_FPS_wpmap = pd.concat([PML_FPS_pmap, work_rates], axis=1).drop(
    columns=['work_att','work_def','work_rate','player_positions','body_type']).copy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()

In [ ]:
PML_FPS_prefoot = PML_FPS_wpmap[['preferred_foot']].copy()

In [ ]:
prefoot_1hot = cat_encoder.fit_transform(PML_FPS_prefoot)
prefoot_1hot

In [ ]:
cat_encoder.categories_

In [ ]:
PML_FPS_wpmap.info()

In [ ]:
prefoot_1hot_df = pd.DataFrame(prefoot_1hot.toarray(), columns=['left','right'])
prefoot_1hot_df = prefoot_1hot_df.astype(int)
PML_FPS_wpmap = PML_FPS_wpmap.reset_index() 
PML_FPS_fwpmap = pd.concat([PML_FPS_wpmap, prefoot_1hot_df], axis=1).drop(columns=['index','preferred_foot'])
PML_FPS_fwpmap.head(2)

In [ ]:
PML_FPS_ha = PML_FPS_fwpmap[['Home_Away']].copy()
PML_FPS_ha.info()

In [ ]:
home_away_1hot = cat_encoder.fit_transform(PML_FPS_ha)
home_away_1hot

In [ ]:
cat_encoder.categories_

In [ ]:
home_away_1hot_arr = pd.DataFrame(home_away_1hot.toarray(), columns=['away','home'])

PML_FPS_hfwpmap = pd.concat([PML_FPS_fwpmap, home_away_1hot_arr], axis=1).drop(columns =["Home_Away"])
PML_FPS_hfwpmap

In [ ]:
PML_FPS_hfwpmap.info()

In [ ]:
#PML_FPS_hfwpmap[['Date','club']].unique().tolist()
#PML_FPS_hfwpmap.query('Date == "25.10.2014 06:45" and club == "Manchester City"')

#### Asignamos un match_id por partido sabiendo que estan en orden de 20 en 20 (no recuerdo como lo hice antes usando "Date")

In [ ]:
match_id = []
for i in range(380):
    for _ in range(20):
        match_id.append(i)
match_id_df = pd.DataFrame(match_id, columns = ['match_id'])
match_id_df

In [ ]:
PML_FPS_ihfwpmap = pd.concat([match_id_df, PML_FPS_hfwpmap], axis=1)
PML_FPS_ihfwpmap.tail(1)

In [ ]:
PML_FPS_ihfwpmap.to_csv('PML_FPS.csv', index=False)

In [ ]:
#matches_and_players.to_csv('matches_and_players.csv', index=False)
#matches_PL.to_csv('matches_PL.csv', index=False)
#players_id_drop_rows.to_csv('players_id_drop_rows.csv', index=False)

### Hare otros 2 archivos que tenga el score home en uno y score away en otro. Y la dif de goles.